<a href="https://colab.research.google.com/github/codesamskaaraH/RNN/blob/main/Wiz_of_OZ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install datasets

In [1]:
# from datasets import load_dataset
# dataset = load_dataset("pg19")

In [57]:
# Alice in wonderland book
import requests
content = requests.get("https://www.gutenberg.org/files/55/55-0.txt").text
with open("wonderful_wiz.txt", "w", encoding="utf-8") as f:
  f.write(content)

In [58]:
with open('wonderful_wiz.txt') as f:
    data = f.readlines()

In [59]:
s_line = data[3]

In [60]:
with open('wonderful_wiz_norm.txt','w',encoding="utf-8") as f:
    f.write(s_line + '/n')
    for line in data:
        f.write(line)

In [61]:
print(content[:100])

ï»¿The Project Gutenberg eBook of The Wonderful Wizard of Oz, by L. Frank Baum

This eBook is for 


In [62]:
from string import punctuation

import numpy as np
from sklearn.model_selection import train_test_split

import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torch.nn import functional as F

In [63]:
sequence_length = 100
batch_size = 128

# dataset file path
FILE_PATH = "wonderful_wiz_norm.txt"

# read the data
text = open(FILE_PATH, encoding="utf-8").read()

# remove caps, comment this code if you want uppercase characters as well
text = text.lower()
# remove punctuation
text = text.translate(str.maketrans("", "", punctuation))


In [64]:
def strip_non_ascii(string):
    ''' Returns the string without non ASCII characters'''
    stripped = (c for c in string if 0 < ord(c) < 127)
    return ''.join(stripped)

In [65]:
text = strip_non_ascii(text)

In [66]:
text[:100]

'most other parts of the world at no cost and with almost no restrictions\nnthe project gutenberg eboo'

In [67]:
# print some stats
n_chars = len(text)
vocab = ''.join(sorted(set(text)))
print("unique_chars:", vocab)
n_unique_chars = len(vocab)
print("Number of characters:", n_chars)
print("Number of unique characters:", n_unique_chars)

unique_chars: 
 0123456789abcdefghijklmnopqrstuvwxyz
Number of characters: 218948
Number of unique characters: 38


In [68]:
print('\ufeff')

﻿


In [69]:
vocab

'\n 0123456789abcdefghijklmnopqrstuvwxyz'

In [70]:
for i, c in enumerate("abc"):
  print(i, c)

0 a
1 b
2 c


In [71]:
# dictionary that converts characters to integers
char2int = {c: i for i, c in enumerate(vocab)}
# dictionary that converts integers to characters
int2char = {i: c for i, c in enumerate(vocab)}

In [72]:
print(char2int)

{'\n': 0, ' ': 1, '0': 2, '1': 3, '2': 4, '3': 5, '4': 6, '5': 7, '6': 8, '7': 9, '8': 10, '9': 11, 'a': 12, 'b': 13, 'c': 14, 'd': 15, 'e': 16, 'f': 17, 'g': 18, 'h': 19, 'i': 20, 'j': 21, 'k': 22, 'l': 23, 'm': 24, 'n': 25, 'o': 26, 'p': 27, 'q': 28, 'r': 29, 's': 30, 't': 31, 'u': 32, 'v': 33, 'w': 34, 'x': 35, 'y': 36, 'z': 37}


In [73]:
print(int2char)

{0: '\n', 1: ' ', 2: '0', 3: '1', 4: '2', 5: '3', 6: '4', 7: '5', 8: '6', 9: '7', 10: '8', 11: '9', 12: 'a', 13: 'b', 14: 'c', 15: 'd', 16: 'e', 17: 'f', 18: 'g', 19: 'h', 20: 'i', 21: 'j', 22: 'k', 23: 'l', 24: 'm', 25: 'n', 26: 'o', 27: 'p', 28: 'q', 29: 'r', 30: 's', 31: 't', 32: 'u', 33: 'v', 34: 'w', 35: 'x', 36: 'y', 37: 'z'}


In [74]:
text[5000:6000]

'ys merry voice reached\nher ears and she still looked at the little girl with wonder that she\ncould find anything to laugh at\n\nuncle henry never laughed he worked hard from morning till night and\ndid not know what joy was he was gray also from his long beard to his\nrough boots and he looked stern and solemn and rarely spoke\n\nit was toto that made dorothy laugh and saved her from growing as gray\nas her other surroundings toto was not gray he was a little black\ndog with long silky hair and small black eyes that twinkled merrily on\neither side of his funny wee nose toto played all day long and\ndorothy played with him and loved him dearly\n\ntoday however they were not playing uncle henry sat upon the\ndoorstep and looked anxiously at the sky which was even grayer than\nusual dorothy stood in the door with toto in her arms and looked at\nthe sky too aunt em was washing the dishes\n\nfrom the far north they heard a low wail of the wind and uncle henry\nand dorothy could see wher

In [75]:
# convert all text into integers
encoded_text = np.array([char2int[c] for c in text])

In [76]:
len(encoded_text), len(text)

(218948, 218948)

In [77]:
text[:25]

'most other parts of the w'

In [78]:
encoded_text[:25]

array([24, 26, 30, 31,  1, 26, 31, 19, 16, 29,  1, 27, 12, 29, 31, 30,  1,
       26, 17,  1, 31, 19, 16,  1, 34])

In [79]:
# tokenize  input_seq, target_seq
def create_sequence_data(text, sequence_length):
  input_seq = []
  target_seq = []
  for idx in range(0, len(text), sequence_length):
    st_idx = idx
    end_idx = st_idx + sequence_length + 1
    if end_idx > len(text):
      # Exclude last slice that may not be full
      continue
    input_seq.append(text[st_idx:end_idx-1])
    target_seq.append(text[st_idx+1:end_idx])
  return input_seq, target_seq

In [80]:
input_seq, target_seq = create_sequence_data(encoded_text, sequence_length=sequence_length)

In [81]:
input_seq[0:3]

[array([24, 26, 30, 31,  1, 26, 31, 19, 16, 29,  1, 27, 12, 29, 31, 30,  1,
        26, 17,  1, 31, 19, 16,  1, 34, 26, 29, 23, 15,  1, 12, 31,  1, 25,
        26,  1, 14, 26, 30, 31,  1, 12, 25, 15,  1, 34, 20, 31, 19,  1, 12,
        23, 24, 26, 30, 31,  1, 25, 26,  1, 29, 16, 30, 31, 29, 20, 14, 31,
        20, 26, 25, 30,  0, 25, 31, 19, 16,  1, 27, 29, 26, 21, 16, 14, 31,
         1, 18, 32, 31, 16, 25, 13, 16, 29, 18,  1, 16, 13, 26, 26]),
 array([22,  1, 26, 17,  1, 31, 19, 16,  1, 34, 26, 25, 15, 16, 29, 17, 32,
        23,  1, 34, 20, 37, 12, 29, 15,  1, 26, 17,  1, 26, 37,  1, 13, 36,
         1, 23,  1, 17, 29, 12, 25, 22,  1, 13, 12, 32, 24,  0,  0, 31, 19,
        20, 30,  1, 16, 13, 26, 26, 22,  1, 20, 30,  1, 17, 26, 29,  1, 31,
        19, 16,  1, 32, 30, 16,  1, 26, 17,  1, 12, 25, 36, 26, 25, 16,  1,
        12, 25, 36, 34, 19, 16, 29, 16,  1, 20, 25,  1, 31, 19, 16]),
 array([ 1, 32, 25, 20, 31, 16, 15,  1, 30, 31, 12, 31, 16, 30,  1, 12, 25,
        15,  0, 24, 26, 

In [82]:
target_seq[0:3]

[array([26, 30, 31,  1, 26, 31, 19, 16, 29,  1, 27, 12, 29, 31, 30,  1, 26,
        17,  1, 31, 19, 16,  1, 34, 26, 29, 23, 15,  1, 12, 31,  1, 25, 26,
         1, 14, 26, 30, 31,  1, 12, 25, 15,  1, 34, 20, 31, 19,  1, 12, 23,
        24, 26, 30, 31,  1, 25, 26,  1, 29, 16, 30, 31, 29, 20, 14, 31, 20,
        26, 25, 30,  0, 25, 31, 19, 16,  1, 27, 29, 26, 21, 16, 14, 31,  1,
        18, 32, 31, 16, 25, 13, 16, 29, 18,  1, 16, 13, 26, 26, 22]),
 array([ 1, 26, 17,  1, 31, 19, 16,  1, 34, 26, 25, 15, 16, 29, 17, 32, 23,
         1, 34, 20, 37, 12, 29, 15,  1, 26, 17,  1, 26, 37,  1, 13, 36,  1,
        23,  1, 17, 29, 12, 25, 22,  1, 13, 12, 32, 24,  0,  0, 31, 19, 20,
        30,  1, 16, 13, 26, 26, 22,  1, 20, 30,  1, 17, 26, 29,  1, 31, 19,
        16,  1, 32, 30, 16,  1, 26, 17,  1, 12, 25, 36, 26, 25, 16,  1, 12,
        25, 36, 34, 19, 16, 29, 16,  1, 20, 25,  1, 31, 19, 16,  1]),
 array([32, 25, 20, 31, 16, 15,  1, 30, 31, 12, 31, 16, 30,  1, 12, 25, 15,
         0, 24, 26, 30, 

In [83]:
train_seq, valid_seq, train_targets, valid_targets = train_test_split(input_seq, target_seq, test_size=0.1)

In [84]:
len(train_seq), len(valid_seq)

(1970, 219)

In [85]:
def one_hot_encode(sequence, vocab_size):
    # Creating a multi-dimensional array of zeros with the desired output shape
    # (Sequence Length, One-Hot Encoding Size)
    seq_length = len(sequence)
    output = np.zeros((seq_length, vocab_size), dtype=np.float32)

    for seq in range(seq_length):
      output[seq, sequence[seq]] = 1
    
    return output

In [86]:
one_hot_encode([1, 2, 26], len(vocab)).shape

(3, 38)

In [87]:
class TextGenDataset(Dataset):
  def __init__(self, text_seq, target_seq, seq_length, vocab_size):
    super().__init__()
    self.text_seq = text_seq
    self.target_seq = target_seq
    self.seq_length = seq_length
    self.vocab_size = vocab_size

  def __getitem__(self, idx):
    return one_hot_encode(self.text_seq[idx], self.vocab_size), self.target_seq[idx]

  def __len__(self):
    return len(self.text_seq)

train_ds = TextGenDataset(train_seq, train_targets, sequence_length, len(vocab))
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=False, drop_last=True)

valid_ds = TextGenDataset(valid_seq, valid_targets, sequence_length, len(vocab))
valid_dl = DataLoader(valid_ds, batch_size=batch_size, drop_last=True)


In [88]:
x, y = train_ds[0]
x.shape, y.shape

((100, 38), (100,))

In [89]:
print(x)
print(y)

[[0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[17 29 26 24  1 13 16 19 20 25 15  1 31 19 16 24  1 31 19 16 36  1 31 32
 29 25 16 15  1 12 25 15  1 34 12 23 22 16 15  1 31 19 29 26 32 18 19  1
 31 19 16  1 17 26 29 16 30 31  1 12  1 17 16 34  0 30 31 16 27 30  1 34
 19 16 25  1 15 26 29 26 31 19 36  1 15 20 30 14 26 33 16 29 16 15  1 30
 26 24 16 31]


In [90]:
for x, y in train_dl:
  break

In [91]:
x.shape, y.shape

(torch.Size([128, 100, 38]), torch.Size([128, 100]))

In [92]:
print(x[0][2])

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0.])


In [93]:
class Model(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers):
        super(Model, self).__init__()

        # Defining some parameters
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        #Defining the layers
        # RNN Layer
        self.rnn = nn.GRU(input_size, hidden_dim, n_layers, batch_first=True)   
        # Fully connected layer
        self.fc = nn.Linear(hidden_dim, output_size)
    
    def forward(self, x, hidden=None):
        
        batch_size = x.size(0)
        # Passing in the input and hidden state into the model and obtaining outputs
        out, hidden = self.rnn(x, hidden)
        
        # Reshaping the outputs such that it can be fit into the fully connected layer
        out = out.contiguous().view(-1, self.hidden_dim)
        out = self.fc(out)
        return out, hidden.detach()

In [94]:

# Instantiate the model with hyperparameters
model = Model(input_size=len(vocab), output_size=len(vocab), hidden_dim=512, n_layers=3)

# Define hyperparameters
n_epochs = 100
lr=0.001
device = "cuda" if torch.cuda.is_available() else "cpu"

# Define Loss, Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [95]:
# Training Run
def train(model, train_dl, criterion, optimizer, batch_size, device=None, n_epochs=100):
  if device is None:
    device = "cuda" if torch.cuda.is_available() else "cpu"
  model = model.to(device)
  model.train()
  criterion = criterion.to(device)
  for epoch in range(1, n_epochs + 1):
    hidden = None
    n_batches = 0
    losses = 0
    for input_seq, target_seq in train_dl:
      input_seq, target_seq = input_seq.to(device), target_seq.to(device)
      optimizer.zero_grad() # Clears existing gradients from previous epoch
      output, hidden = model(input_seq, hidden)
      loss = criterion(output, target_seq.view(-1).long())
      loss.backward() # Does backpropagation and calculates gradients
      optimizer.step() # Updates the weights accordingly
      # hidden.detach_()
      losses += loss.item()
      n_batches += 1
    
    if epoch%10 == 0:
      print('Epoch: {}/{}.............'.format(epoch, n_epochs), end=' ')
      print("Loss: {:.4f}".format(losses / n_batches))
  return model

In [96]:
model = train(model, train_dl, criterion, optimizer, batch_size, device, n_epochs=200)

Epoch: 10/200............. Loss: 1.8126
Epoch: 20/200............. Loss: 1.2848
Epoch: 30/200............. Loss: 1.0706
Epoch: 40/200............. Loss: 0.9242
Epoch: 50/200............. Loss: 0.7577
Epoch: 60/200............. Loss: 0.6117
Epoch: 70/200............. Loss: 0.5762
Epoch: 80/200............. Loss: 0.4005
Epoch: 90/200............. Loss: 0.3482
Epoch: 100/200............. Loss: 0.2285
Epoch: 110/200............. Loss: 0.2168
Epoch: 120/200............. Loss: 0.1130
Epoch: 130/200............. Loss: 0.0646
Epoch: 140/200............. Loss: 0.0340
Epoch: 150/200............. Loss: 0.0205
Epoch: 160/200............. Loss: 0.0107
Epoch: 170/200............. Loss: 0.0086
Epoch: 180/200............. Loss: 0.0072
Epoch: 190/200............. Loss: 0.0062
Epoch: 200/200............. Loss: 0.0054


In [97]:
#model.load_state_dict(torch.load("model.pt"))

In [98]:
def predict(model, hidden, character, char2int, int2char, device):  
    # One-hot encoding our input to fit into the model
    # print(character)
    character = np.array([char2int[c] for c in character])
    # print(character)
    character = one_hot_encode(character, vocab_size=len(char2int))
    # print(character.shape)
    character = torch.from_numpy(character).unsqueeze(0).to(device)
    with torch.no_grad():
      out, hidden = model(character, hidden)
    # print(hidden.size())
    prob = nn.functional.softmax(out[-1], dim=0).data
    # Taking the class with the highest probability score from the output
    char_ind = torch.max(prob, dim=0)[1].item()

    return int2char[char_ind], hidden

In [99]:
def sample(model, char2int, int2char, out_len, start='hey', device=None):
    model.eval() # eval mode
    if device is None:
      device = "cuda" if torch.cuda.is_available() else "cpu"    
    model.to(device)
    start = start.lower()
    # First off, run through the starting characters
    chars = [ch for ch in start]
    size = out_len - len(chars)
    # Now pass in the previous characters and get a new one
    hidden = None
    for ii in range(size):
        # print(chars)
        char, hidden = predict(model, hidden, chars[-1], char2int, int2char, device)
        chars.append(char)

    return ''.join(chars)

In [100]:
sample(model, char2int, int2char, out_len=150, start="Oh gracious")

'oh graciousend me back to kansas\n\nwhere is the emerald city he inquired and who is oz\n\nwhy dont you know she returned and at once the winged monkeys\nc'

In [50]:
#whirlwinds arose
sample(model, char2int, int2char, out_len=150, start="whirlwinds arose")

'whirlwinds arosed t\nmind t\nmind t\nmind t\nmind t\nmind t\nmind t\nmind t\nmind t\nmind t\nmind t\nmind t\nmind t\nmind t\nmind t\nmind t\nmind t\nmind t\nmind t\nmind'

In [47]:
sample(model, char2int, int2char, out_len=150, start="Uncle Henry")

'uncle henry around her but dorothy\nfound she was riding quite easily after the field mice and how they had generously saved\nhim from death and the cow'

In [ ]:
torch.save(model.state_dict(), "./model.pt")